In [1]:
import io
import logging
import subprocess
import sys
import warnings
from contextlib import suppress
from datetime import timedelta
from functools import reduce
from logging import StreamHandler
from os import chdir, getcwd, makedirs
from os.path import dirname, exists, join
from subprocess import CalledProcessError, check_output
from typing import List

import humanize
import numpy as np
import pandas as pd
import pandas_ta as ta
from google import auth
from google.cloud import bigquery
from moonshot.strategies.base import Moonshot
from numpy import nan
from pandas.testing import assert_frame_equal, assert_series_equal

if exists('/codeload'):
    WORK_DIR = '/codeload'
else:
    base_dir = check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode()
    WORK_DIR = join(base_dir, 'src')
    sys.path.extend([base_dir,
                    join(base_dir, 'src'),
                    join(base_dir, 'notebook'),
                    join(base_dir, 'notebook/*')])

chdir(WORK_DIR)

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.8f}'.format

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(StreamHandler())

In [2]:
%set_env GOOGLE_APPLICATION_CREDENTIALS $base_dir/credentials/service_account.json

CREDENTIALS, PROJECT_ID = auth.default()
REGION = 'asia-southeast1'

%set_env GCLOUD_PROJECT $PROJECT_ID
%set_env REGION $REGION

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/trung/Projects/tkteam/py-datasource/credentials/service_account.json
env: GCLOUD_PROJECT=trading-r-and-d
env: REGION=asia-southeast1


In [3]:
from data_providers.data_provider import DataProvider
from data_providers.tradingview_provider import TradingViewProvider

In [4]:
provider: DataProvider = TradingViewProvider()
provider.TOKEN = ''
provider.WORKERS_NO = 4

In [4]:
# symbols = [
#     'SP:S5COND',
# ]

# fields = [
#     'description'
#     'price',
#     'change_5d',
#     'change_5d_pct',
#     'low_5d',
#     'high_5d',
# ]

# results = provider.quotes(symbols=symbols, fields=fields)

In [25]:
symbols = [
    'BTCUSD',
]

ohlcv = provider.ohclv(symbols=symbols,
                       freq='1D',
                       total_candles=252 + 3,
                       tzinfo='America/Chicago').reset_index()

In [23]:
ohlcv

,Date,Symbol,Open,High,Low,Close,Volume
0,2024-01-01 18:00:00-06:00,BTCUSD,"44,187.00000000","45,922.00000000","44,167.00000000","44,970.00000000","3,931.57911761"
1,2024-01-02 18:00:00-06:00,BTCUSD,"44,972.00000000","45,510.00000000","41,454.00000000","42,865.00000000","3,980.82805610"
2,2024-01-03 18:00:00-06:00,BTCUSD,"42,863.00000000","44,795.00000000","42,656.00000000","44,186.00000000","2,918.11934919"
3,2024-01-04 18:00:00-06:00,BTCUSD,"44,195.00000000","44,366.00000000","42,500.00000000","44,184.00000000","3,155.46601801"
4,2024-01-05 18:00:00-06:00,BTCUSD,"44,189.00000000","44,253.00000000","43,440.00000000","43,999.00000000",980.53783688
...,...,...,...,...,...,...,...
249,2024-09-06 19:00:00-05:00,BTCUSD,"53,947.00000000","54,839.00000000","53,731.00000000","54,147.00000000","1,046.86393200"
250,2024-09-07 19:00:00-05:00,BTCUSD,"54,150.00000000","55,306.00000000","53,636.00000000","54,866.00000000","1,238.32664936"
251,2024-09-08 19:00:00-05:00,BTCUSD,"54,876.00000000","58,070.00000000","54,587.00000000","57,059.00000000","2,644.10962784"
252,2024-09-09 19:00:00-05:00,BTCUSD,"57,056.00000000","58,042.00000000","56,406.00000000","57,659.00000000","1,878.56149502"


In [12]:
ohlcv = ohlcv.reset_index()

In [15]:
ohlcv.loc[ohlcv.symbol == 'SP:SPX', 'symbol'] = 'SPX'

In [16]:
ohlcv

,timestamp,symbol,open,high,low,close,volume
0,2023-09-07 13:30:00+00:00,NASDAQ:TSLA,245.07000000,252.81000000,243.26500000,251.49000000,"115,312,886.00000000"
1,2023-09-08 13:30:00+00:00,NASDAQ:TSLA,251.22000000,256.52000000,246.67000000,248.50000000,"118,559,635.00000000"
2,2023-09-11 13:30:00+00:00,NASDAQ:TSLA,264.27000000,274.85000000,260.61000000,273.58000000,"174,667,852.00000000"
3,2023-09-12 13:30:00+00:00,NASDAQ:TSLA,270.76000000,278.39000000,266.60000000,267.48000000,"135,999,866.00000000"
4,2023-09-13 13:30:00+00:00,NASDAQ:TSLA,270.07000000,274.98000000,268.10000000,271.30000000,"111,673,737.00000000"
...,...,...,...,...,...,...,...
499,2024-08-30 13:30:00+00:00,SPX,"5,612.74000000","5,651.37000000","5,581.79000000","5,648.39000000","2,627,000,000.00000000"
500,2024-09-03 13:30:00+00:00,SPX,"5,623.89000000","5,623.89000000","5,504.33000000","5,528.92000000","2,809,000,000.00000000"
501,2024-09-04 13:30:00+00:00,SPX,"5,506.68000000","5,552.99000000","5,503.66000000","5,520.08000000","2,428,000,000.00000000"
502,2024-09-05 13:30:00+00:00,SPX,"5,520.08000000","5,546.30000000","5,480.54000000","5,503.42000000","2,367,000,000.00000000"


In [15]:
provider.calc_perf(ohlcv, '24h')

{'NASDAQ:TSLA': {'close': 221.1,
  'change_24h': -1.6200000000000045,
  'change_24h_pct': -0.007273706896551745,
  'low_24h': 219.56,
  'high_24h': 228.22}}

In [7]:
provider.calc_perf(ohlcv, '5D')

{'NASDAQ:TSLA': {'close': 221.08,
  'change_5d': -0.7800000000000011,
  'change_5d_pct': -0.0035157306409447447,
  'low_5d': 221.0,
  'high_5d': 221.93}}

In [9]:
provider.calc_perf(ohlcv, '1M')

{'NASDAQ:TSLA': {'close': 221.08,
  'change_1m': -0.39999999999997726,
  'change_1m_pct': -0.0018060321473721208,
  'low_1m': 221.0,
  'high_1m': 222.47}}

In [8]:
provider.calc_perf(ohlcv, 'MTD')

{'SP:S5COND': {'close': 1463.72,
  'change_mtd': 40.22000000000003,
  'change_mtd_pct': 0.028254302774850738,
  'low_mtd': 1411.67,
  'high_mtd': 1473.95}}

In [26]:
provider.calc_perf(ohlcv, 'YTD')

Y Date
2023   44,187.00000000
2024   56,854.00000000
Freq: Y-DEC, Name: close, dtype: float64 Date
2023               NaN
2024   44,187.00000000
Freq: Y-DEC, Name: close, dtype: float64 Date
2023               NaN
2024   44,187.00000000
Freq: Y-DEC, Name: close_ytd_prev, dtype: float64


{'BTCUSD': {'close': 56854.0,
  'change_ytd': 12667.0,
  'change_ytd_pct': 0.2866680245321022,
  'low_ytd': 38505.0,
  'high_ytd': 73794.0}}